In [ ]:
import exifread
import rasterio
from rasterio.transform import Affine, from_origin
from rasterio.crs import CRS
from pathlib import Path
import os

In [4]:
def fix_georeference(tiff_path, output_path, epsg_code=5234):
    """Add EPSG:5234 CRS and update transform if missing."""
    # Check if the input file exists
    if not os.path.exists(tiff_path):
        raise FileNotFoundError(f"Input file {tiff_path} does not exist.")
    # Check if the output directory exists, create if not
    output_dir = Path(output_path).parent
    if not output_dir.exists():
        os.makedirs(output_dir, exist_ok=True)

    
    with rasterio.open(tiff_path) as src:
        # Read the image data and metadata
        img_data = src.read()
        profile = src.profile
        
        # Force CRS to EPSG:5234 (Sri Lanka)
        profile.update(crs=CRS.from_epsg(epsg_code))
        
        # If transform is identity (1,0,0,0,1,0), set a dummy transform
        if src.transform.is_identity:
            profile.update(transform=Affine(1.0, 0.0, 0.0, 0.0, -1.0, 0.0))  # Flip Y-axis for GIS
        
        # Write the updated GeoTIFF
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(img_data)

# Example usage
# fix_georeference("DJI_0001_R.TIFF", "DJI_0001_R_GEOREF.TIFF")

In [5]:
def extract_metadata(tiff_path):
    # Extract GPS and camera info using exifread
    with open(tiff_path, 'rb') as f:
        tags = exifread.process_file(f)
        lat = tags.get('GPS GPSLatitude')
        lon = tags.get('GPS GPSLongitude')
        focal_length = tags.get('EXIF FocalLength')
        print(f"Lat: {lat}, Lon: {lon}, Focal: {focal_length}")

    # Extract geospatial info using rasterio
    with rasterio.open(tiff_path) as src:
        crs = src.crs  # Coordinate Reference System
        transform = src.transform  # Affine transform
        print(f"CRS: {crs}, Transform: {transform}")
        print("++++++++++++++++++++++++++++++++++++++++++++++")
        # Check if the CRS is None (missing)
        if (crs is None):
            output_path =  os.path.join("test/fix_crs/", tiff_path.name)
            fix_georeference(tiff_path, output_path)
            print(f"Georeference fixed and saved to {output_path}")
            print("New file metadata:")
            with rasterio.open(output_path) as new_src:
                new_crs = new_src.crs
                new_transform = new_src.transform
                print(f"New CRS: {new_crs}, New Transform: {new_transform}")
        else:
            print("No need to fix georeference.")



# Example usage
# E:\Research_topic\Gal_Oya\118FPLAN\odm_orthophoto
# imger_path = Path("../../../../Gal_Oya/108FPLAN/DJI_0025.TIF")
# imger_path = Path("../../../../Gal_Oya/118FPLAN/odm_orthophoto/odm_orthophoto.original.tif")
# imger_path = Path("../../data/FOM_PIX4D/germination/main/germination_main.tif")
imger_path = Path("../../temp/images/image_0009.tif")
if imger_path.exists():
    print(f"File exists: {imger_path}")
else:
    print(f"File does not exist: {imger_path}")

print(f"File name {imger_path.name}  related metadata")
extract_metadata(imger_path)

File exists: ..\..\temp\images\image_0009.tif
File name image_0009.tif  related metadata
Lat: [7, 16, 36407/1250], Lon: [81, 41, 41997/2500], Focal: 287/50
CRS: None, Transform: | 1.00, 0.00, 0.00|
| 0.00, 1.00, 0.00|
| 0.00, 0.00, 1.00|
++++++++++++++++++++++++++++++++++++++++++++++
Georeference fixed and saved to test/fix_crs/image_0009.tif
New file metadata:
New CRS: EPSG:5234, New Transform: | 1.00, 0.00, 0.00|
| 0.00,-1.00, 0.00|
| 0.00, 0.00, 1.00|


c:\Users\user\anaconda3\envs\research_env\lib\site-packages\rasterio\__init__.py:378: NotGeoreferencedWarning: The given matrix is equal to Affine.identity or its flipped counterpart. GDAL may ignore this matrix and save no geotransform without raising an error. This behavior is somewhat driver-specific.
  dataset = writer(


In [33]:
#check bands
with rasterio.open(imger_path) as src:
    print(f"Number of bands: {src.count}")
    print(f"Image dimensions: {src.width} x {src.height}")
    print(f"Image CRS: {src.crs}")
    print(f"Image transform: {src.transform}")
    for i in range(1, src.count + 1):
        band = src.read(i)
        desc = src.descriptions[i - 1]  # descriptions is a tuple, zero-indexed
        print(f"Band {i} description: {desc}")
        # print(f"Band {i} shape: {band.shape}")
        # print(f"Band {i} data type: {band.dtype}")
        # print(f"Band {i} min value: {band.min()}")
        # print(f"Band {i} max value: {band.max()}")
        print("++++++++++++++++++++++++++++++++++++++++++++++")


Number of bands: 6
Image dimensions: 1409 x 2114
Image CRS: EPSG:32644
Image transform: | 0.05, 0.00, 575873.07|
| 0.00,-0.05, 804192.13|
| 0.00, 0.00, 1.00|
Band 1 description: Red
++++++++++++++++++++++++++++++++++++++++++++++
Band 2 description: Green
++++++++++++++++++++++++++++++++++++++++++++++
Band 3 description: Blue
++++++++++++++++++++++++++++++++++++++++++++++
Band 4 description: NIR
++++++++++++++++++++++++++++++++++++++++++++++
Band 5 description: RedEdge
++++++++++++++++++++++++++++++++++++++++++++++
Band 6 description: None
++++++++++++++++++++++++++++++++++++++++++++++
